In [33]:
#Import library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from math import radians, cos, sin, asin, sqrt

In [34]:
#Import dataset
dataset = pd.read_excel('/Users/lucy/Desktop/zoopla/zoop_v13_copy.xlsx')

In [35]:
#Import Traingeo dataset
traingeo = pd.read_excel('/Users/lucy/Desktop/zoopla/traingeo.xlsx')

In [36]:
#Display dataset
dataset.head()

,listing Id,published date,description,outcode,post town,furnished state,property type,bathrooms,bedrooms,num_reception,bills_included,latitude,longitude,monthly rent,dist_school,ofsted_rating_school
0,51756505,2021-08-07 07:35:46,***Available 11th August 2021**Furnished groun...,B1,Birmingham,part_furnished,Flat,1,1,0,0,52.476063,-1.911296,675,0.507,Good
1,59345196,2021-08-02 15:27:21,Three Bedroom Property Available in SparkhillT...,B11,Birmingham,unfurnished,Terraced house,1,3,2,0,52.456490,-1.861211,850,0.066,Good
2,39706692,2021-08-06 17:54:01,This well presented ground floor apartment is ...,B31,Birmingham,unfurnished,Flat,1,2,0,0,52.418476,-1.973989,560,0.206,Good
3,59398269,2021-08-07 07:26:42,Barkers are pleased to offer a self contained ...,LE2,Leicester,part_furnished,Flat,1,1,0,0,52.616234,-1.124677,675,0.404,Good
4,59398270,2021-08-07 07:26:42,Barkers are please to offer a three share mais...,LE3,Leicester,furnished,Flat,1,3,1,0,52.631287,-1.145698,845,0.215,Good


In [37]:
#Display traingeo dataset
traingeo.head()

,station,ELR,Mileage,status,owner,operator,Lon,Lat,Grid Reference
0,Abbey Wood (ABW),NKL,11m 43ch,Open,Network Rail Infrastructure; to transfer on 12...,MTR Corporation (Crossrail) from 22 October 2017,0.1204,51.4908,TQ472790
1,Abbey Wood (ABX),XRS,24.458km,Under construction,TfL,MTR Corporation (Crossrail) from opening,0.1204,51.4908,TQ472790
2,Aber (ABE),CAR,8m 69ch,NaN,Transport for Wales from 28 March 2020,Transport for Wales Rail from 7 February 2021,-3.2305,51.5755,ST148869
3,Abercynon (ACY) formerly Abercynon South,CAM ABD,16m 28ch 16m 28ch,NaN,Transport for Wales from 28 March 2020,Transport for Wales Rail from 7 February 2021,-3.3294,51.6434,ST081947
4,Abercynon North (ABR),ABD,16m 40ch,Closed,NaN,Arriva Trains Wales/Trenau Arriva Cymru from 7...,-3.3275,51.6454,ST082948


In [38]:
#Define a function to calculate distance in kms between co ordinates from each dataset

def dist(lat1, long1, lat2, long2):
    """
Replicating the same formula as mentioned in Wiki
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [39]:
#Find the nearest station using the function from the above code
def find_nearest(lat, long):
    distances = traingeo.apply(
        lambda row: dist(lat, long, row['Lat'], row['Lon']), 
        axis=1)
    return traingeo.loc[distances.idxmin(), 'station']

In [40]:
#get latitude and longitude of nearest station and apply to dataset
dataset['station'] = dataset.apply(
    lambda row: find_nearest(row['latitude'], row['longitude']), 
    axis=1)

In [41]:
#merge the nearest station name, lat and long with dataset
dataset = pd.merge(dataset,traingeo[['station','Lat','Lon']],on='station', how='left')

In [42]:
#Dataset shows the nearest station with its lat and long for each instance
dataset.head(3)

,listing Id,published date,description,outcode,post town,furnished state,property type,bathrooms,bedrooms,num_reception,bills_included,latitude,longitude,monthly rent,dist_school,ofsted_rating_school,station,Lat,Lon
0,51756505,2021-08-07 07:35:46,***Available 11th August 2021**Furnished groun...,B1,Birmingham,part_furnished,Flat,1,1,0,0,52.476063,-1.911296,675,0.507,Good,Five Ways (FWY),52.4712,-1.9132
1,59345196,2021-08-02 15:27:21,Three Bedroom Property Available in SparkhillT...,B11,Birmingham,unfurnished,Terraced house,1,3,2,0,52.456490,-1.861211,850,0.066,Good,Small Heath (SMA),52.4638,-1.8593
2,39706692,2021-08-06 17:54:01,This well presented ground floor apartment is ...,B31,Birmingham,unfurnished,Flat,1,2,0,0,52.418476,-1.973989,560,0.206,Good,Northfield (NFD),52.4082,-1.9657


In [43]:
#To calculate the actual distance in Kms

#Import radians, cos, sin, asin, sqrt
from math import radians, cos, sin, asin, sqrt

#Define a function to calculate the distance using the haversine formula
def haversine(lon1, lat1, lon2, lat2):
    
    #Calculate the great circle distance between two points on the earth (specified in decimal degrees)
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Create a new column to generate the output by passing lat long information to Haversine Equation
dataset['dist_rail'] = [haversine(dataset.longitude[i],dataset.latitude[i],dataset.Lon[i],dataset.Lat[i]) for i in range(len(dataset))]
dataset['dist_rail'] = dataset['dist_rail'].round(decimals=3)
# Printing the dataset that displays the distance to the nearest station in kms 
dataset.head(3)

,listing Id,published date,description,outcode,post town,furnished state,property type,bathrooms,bedrooms,num_reception,bills_included,latitude,longitude,monthly rent,dist_school,ofsted_rating_school,station,Lat,Lon,dist_rail
0,51756505,2021-08-07 07:35:46,***Available 11th August 2021**Furnished groun...,B1,Birmingham,part_furnished,Flat,1,1,0,0,52.476063,-1.911296,675,0.507,Good,Five Ways (FWY),52.4712,-1.9132,0.556
1,59345196,2021-08-02 15:27:21,Three Bedroom Property Available in SparkhillT...,B11,Birmingham,unfurnished,Terraced house,1,3,2,0,52.456490,-1.861211,850,0.066,Good,Small Heath (SMA),52.4638,-1.8593,0.823
2,39706692,2021-08-06 17:54:01,This well presented ground floor apartment is ...,B31,Birmingham,unfurnished,Flat,1,2,0,0,52.418476,-1.973989,560,0.206,Good,Northfield (NFD),52.4082,-1.9657,1.273


In [44]:
#Drop the station name, Lat, Lon and keep only the distance column
dataset = dataset.drop(["station", 'Lat', 'Lon'], axis=1)

In [45]:
dataset.head(2)

,listing Id,published date,description,outcode,post town,furnished state,property type,bathrooms,bedrooms,num_reception,bills_included,latitude,longitude,monthly rent,dist_school,ofsted_rating_school,dist_rail
0,51756505,2021-08-07 07:35:46,***Available 11th August 2021**Furnished groun...,B1,Birmingham,part_furnished,Flat,1,1,0,0,52.476063,-1.911296,675,0.507,Good,0.556
1,59345196,2021-08-02 15:27:21,Three Bedroom Property Available in SparkhillT...,B11,Birmingham,unfurnished,Terraced house,1,3,2,0,52.456490,-1.861211,850,0.066,Good,0.823
